In [38]:
%pip install pandas seaborn matplotlib
#!pip install pandas seaborn matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
%pip install gdown
import gdown
url_csv = "https://drive.google.com/drive/folders/1pFuFpNtd6oh2w6Ga7PRlt0kjjwbDjXTP?usp=sharing"
url_embeddings = "https://drive.google.com/drive/folders/1Eop8xnD8XPh5nU0omBgoQ84l7u2W_KQZ?usp=sharing"
gdown.download_folder(url_csv, quiet=True)
gdown.download_folder(url_embeddings, quiet=True)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


['c:\\Users\\misse\\Projects\\pharmahacks_25\\pharmahacks-25\\embeddings\\all_seqs.pkl',
 'c:\\Users\\misse\\Projects\\pharmahacks_25\\pharmahacks-25\\embeddings\\base_seq.pkl',
 'c:\\Users\\misse\\Projects\\pharmahacks_25\\pharmahacks-25\\embeddings\\global_embeddings_map.pkl',
 'c:\\Users\\misse\\Projects\\pharmahacks_25\\pharmahacks-25\\embeddings\\global_embeddings_max_length.npy']

In [41]:
table1 = pd.read_csv('csv_tables/table-1.csv')
table2 = pd.read_csv('csv_tables/table-2.csv')
table3 = pd.read_csv('csv_tables/table-3.csv')

# extract compound names
compound_columns = table3.columns[3:]

# merge compound columns into rows with the corresponding kinase
kinase_compound_kd = pd.melt(
    table3,
    id_vars=['Accession Number', 'Entrez Gene Symbol', 'Kinase'],
    value_vars=compound_columns,
    var_name='Compound',
    value_name='Kd'
)

# merge tables
merged_df = kinase_compound_kd.merge(table1, on=['Accession Number', 'Entrez Gene Symbol', 'Kinase'], how='left')
merged_df = merged_df.merge(table2[['Compound', 'Binding Mode (based on ABL1-phos. vs. -nonphos affinity)', 'S(300nM)', 'S(3000nM)']], on='Compound', how='left')

print(merged_df.head())

  Accession Number Entrez Gene Symbol                         Kinase  \
0      NP_055726.3               AAK1                           AAK1   
1      NP_005148.2               ABL1     ABL1(E255K)-phosphorylated   
2      NP_005148.2               ABL1  ABL1(F317I)-nonphosphorylated   
3      NP_005148.2               ABL1     ABL1(F317I)-phosphorylated   
4      NP_005148.2               ABL1  ABL1(F317L)-nonphosphorylated   

   Compound       Kd Mutant Kinase Group  \
0  A-674563     43.0     NO        Other   
1  A-674563  10001.0    YES           TK   
2  A-674563  10001.0    YES           TK   
3  A-674563  10001.0    YES           TK   
4  A-674563  10001.0    YES           TK   

  Binding Mode (based on ABL1-phos. vs. -nonphos affinity)  S(300nM)  \
0                                       undetermined          0.1166   
1                                       undetermined          0.1166   
2                                       undetermined          0.1166   
3             

In [45]:
relevant_df = merged_df[['S(300nM)', 'S(3000nM)', 'Binding Mode (based on ABL1-phos. vs. -nonphos affinity)', 'Kinase Group', 'Mutant']]
print(relevant_df.head())

   S(300nM)  S(3000nM)  \
0    0.1166     0.2772   
1    0.1166     0.2772   
2    0.1166     0.2772   
3    0.1166     0.2772   
4    0.1166     0.2772   

  Binding Mode (based on ABL1-phos. vs. -nonphos affinity) Kinase Group Mutant  
0                                       undetermined              Other     NO  
1                                       undetermined                 TK    YES  
2                                       undetermined                 TK    YES  
3                                       undetermined                 TK    YES  
4                                       undetermined                 TK    YES  


In [46]:
relevant_df_encoded = pd.get_dummies(relevant_df, columns=['Binding Mode (based on ABL1-phos. vs. -nonphos affinity)', 'Kinase Group', 'Mutant'])
print(relevant_df_encoded.head())

   S(300nM)  S(3000nM)  \
0    0.1166     0.2772   
1    0.1166     0.2772   
2    0.1166     0.2772   
3    0.1166     0.2772   
4    0.1166     0.2772   

   Binding Mode (based on ABL1-phos. vs. -nonphos affinity)_Type I  \
0                                              False                 
1                                              False                 
2                                              False                 
3                                              False                 
4                                              False                 

   Binding Mode (based on ABL1-phos. vs. -nonphos affinity)_Type II  \
0                                              False                  
1                                              False                  
2                                              False                  
3                                              False                  
4                                              False               

In [ ]:
correlation_matrix = relevant_df_encoded.corr()


S(300nM)                                                                 1.000000e+00
S(3000nM)                                                                9.301859e-01
Binding Mode (based on ABL1-phos. vs. -nonphos affinity)_Type I          3.808686e-01
Mutant_NO                                                                1.523698e-15
Kinase Group_TK                                                          7.643670e-16
Kinase Group_Other                                                       1.848379e-16
Mutant_YES                                                               1.137241e-16
Kinase Group_STE                                                         1.090483e-16
Kinase Group_Pathogen                                                    3.477667e-17
Kinase Group_AGC                                                         2.026712e-17
Kinase Group_CMGC                                                       -3.490084e-17
Kinase Group_CK1                                      

In [50]:
print(correlation_matrix['S(3000nM)'].sort_values(ascending=False))


S(3000nM)                                                                1.000000e+00
S(300nM)                                                                 9.301859e-01
Binding Mode (based on ABL1-phos. vs. -nonphos affinity)_Type I          5.038162e-01
Mutant_NO                                                                1.849854e-15
Kinase Group_TK                                                          7.689876e-16
Kinase Group_Other                                                       2.055961e-16
Mutant_YES                                                               1.283630e-16
Kinase Group_STE                                                         9.338511e-17
Kinase Group_Pathogen                                                    4.014719e-17
Kinase Group_AGC                                                        -2.488369e-17
Kinase Group_CMGC                                                       -6.454334e-17
Kinase Group_CK1                                      

In [ ]:
plt.figure(figsize=(12, 8))
